___

#### Data Collection: Weather and City Names
___
<h3 align="Left">by Mohammd Aboutorabi 99242085,Fatemeh Rahideh 99242075,Mohammad Yousofvand 99242156</h3> 

___



## Collecting Weather Data and City Names
---


In order to create a competent recommender system, I need information from cities in the USA. We will then analyze such data, select the features we want to use, and begin building!

----
**Which cities will I include?**

The cities to be included are those that are listed in <a href='https://www.usclimatedata.com/climate/united-states/us'>this</a> website. 


**_States:_** 

<a><img src='https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Images/USA_States_Table_WSite.PNG' style='width: 30em;' align=auto /></a>

As you can observe in the image, the website contains a table that references all the states in the US. 

<a><img src='https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Images/States_Cities_Table_WSite.PNG' style='width: 30em;' align=auto /></a>

Once you click on one of the states, like Alabama in this example, a table of all the cities in such state appears. 

Therefore, we will user this website and BeautifulSoup to get all the cities in each state, and the references (URLs) to each of the cities in each state so we can loop through these and scrap all the weather data.


----


#### Import Libraries

---
The libraries imported are not all used in this notebook. However, to be able to follow the whole project, please make sure you have these installed!

Otherwise use pip or conda to install them on your laptop or computer.

In [12]:
import pandas as pd # import pandas for dataframes
import numpy as np
import requests 
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import re
import matplotlib.pyplot as plt

from IPython.display import HTML, display
from IPython.display import Image 
from IPython.core.display import HTML 

pd.options.display.max_columns = None
pd.options.display.max_rows=None


%matplotlib inline

### Getting List of Cities

---

To get the list of cities, we will be be using <a href='https://www.usclimatedata.com/climate/united-states/us'>this</a> weather website. This will allow is to get the list of states and the names of each city in every state along with the weather data for each city in each state.
1. Get the name of each state along with the link to the websites that contains all the cities in each state. 
2. Get the name of each city along with the link to the websites that contains all the weather information in each city.
3. Scrap the desired weather data for each city in every state as specified in this website.
    - Average monthly temperature
    - Swing from average high to average low temperature for every month.

#### Generating the URLs

In [13]:
# Create list of tuples containing the name of the state and the href
def get_state_names(url):
    states = []
    req = requests.get(url)
    soup = BeautifulSoup(req.content, features='lxml')
    table2 = soup.find_all("a",{"class":"stretched-link"})
    for j in range(0,len(table2)):
        newurl = 'https://www.usclimatedata.com'+table2[j].get('href')
        name_state = table2[j].get_text()
        tempdic = {'State':name_state,'url':newurl}
        states.append(tempdic)
    states_df = pd.DataFrame.from_dict(states)
    return states_df
states_names_list = get_state_names('https://www.usclimatedata.com/')

District of Columbia is weirdly defined in our dataframe... lets fix it.

In [14]:
states_names_list.at[8,'State'] = "District of Columbia"
states_names_list[0:5]

State                                                url
0     Alabama  https://www.usclimatedata.com/climate/alabama/...
1      Alaska  https://www.usclimatedata.com/climate/alaska/u...
2     Arizona  https://www.usclimatedata.com/climate/arizona/...
3    Arkansas  https://www.usclimatedata.com/climate/arkansas...
4  California  https://www.usclimatedata.com/climate/californ...

In [15]:
def get_city_names(url):
    cities_st = []
    req = requests.get(url)
    soup = BeautifulSoup(req.content, features='lxml')
    name_state = soup.find("p",{"class":"selection_title"})
    name_state = name_state.get_text()[:-24]
    table1 = soup.find_all("a", {"class":"stretched-link"})
    for i in range(0,len(table1)):
        name_city = table1[i].get_text()
        newurl = 'https://www.usclimatedata.com'+table1[i].get('href')
        tempdic = {'State':name_state,'City':name_city,'url_city':newurl}
        cities_st.append(tempdic)
    cities_df = pd.DataFrame(cities_st)
    return cities_df
excities = get_city_names('https://www.usclimatedata.com/climate/alabama/united-states/3170')
excities.head()

State            City                                           url_city
0  Alabama         Addison  https://www.usclimatedata.com/climate/addison/...
1  Alabama       Alabaster  https://www.usclimatedata.com/climate/alabaste...
2  Alabama  Alexander City  https://www.usclimatedata.com/climate/alexande...
3  Alabama      Aliceville  https://www.usclimatedata.com/climate/alicevil...
4  Alabama       Andalusia  https://www.usclimatedata.com/climate/andalusi...

In [16]:
state_city_urls = []
for i, row in states_names_list.iterrows():
    random_state_city = get_city_names(row.url)
    state_city_urls.append(random_state_city)


In [17]:
!pip list

Package                      Version
---------------------------- ----------
absl-py                      1.4.0
asttokens                    2.2.1
astunparse                   1.6.3
async-generator              1.10
attrs                        23.1.0
backcall                     0.2.0
beautifulsoup4               4.12.2
branca                       0.6.0
bs4                          0.0.1
cachetools                   5.3.0
certifi                      2023.5.7
cffi                         1.15.1
charset-normalizer           3.1.0
colorama                     0.4.6
comet-ml                     3.33.3
comm                         0.1.3
concat                       0.1.2.dev1
configobj                    5.0.8
contourpy                    1.0.7
cycler                       0.11.0
debugpy                      1.6.7
decorator                    5.1.1
drop                         0.4.1
dulwich                      0.21.5
everett                      3.1.0
exceptiongroup               1.1.1


In [18]:
import pandas as pd


state_city_df=pd.DataFrame()
for i in range(0,len(state_city_urls)):
    ith_df = state_city_urls[i]
    state_city_df = pd.concat([state_city_df, ith_df], ignore_index=True)
state_city_df.head()

State            City                                           url_city
0  Alabama         Addison  https://www.usclimatedata.com/climate/addison/...
1  Alabama       Alabaster  https://www.usclimatedata.com/climate/alabaste...
2  Alabama  Alexander City  https://www.usclimatedata.com/climate/alexande...
3  Alabama      Aliceville  https://www.usclimatedata.com/climate/alicevil...
4  Alabama       Andalusia  https://www.usclimatedata.com/climate/andalusi...

In [19]:
print(state_city_df.shape)
state_city_df.head()

(5850, 3)


State            City                                           url_city
0  Alabama         Addison  https://www.usclimatedata.com/climate/addison/...
1  Alabama       Alabaster  https://www.usclimatedata.com/climate/alabaste...
2  Alabama  Alexander City  https://www.usclimatedata.com/climate/alexande...
3  Alabama      Aliceville  https://www.usclimatedata.com/climate/alicevil...
4  Alabama       Andalusia  https://www.usclimatedata.com/climate/andalusi...

---

Now that we have a dataframe with all the cities and their respective URL to scrap weather data, lets begin scraping!

In [20]:
#Now, let's get the average highs, lows and precipitation in each city
# def get_weather(state, city, url):
#     req = requests.get(url)
#     soup = BeautifulSoup(req.content, features='lxml')
#     temp_high = soup.find_all("td",{"class":"high text-right"})
#     temp_high = [i.get_text() for i in temp_high]
#     temp_high = temp_high[0:12]
#     temp_low = soup.find_all("td",{"class":"low text-right"})
#     temp_low = [i.get_text() for i in temp_low]
#     temp_low = temp_low[0:12]
#     precip = soup.find_all("td",{"class":"text-right"})
#     precip = [i.get_text() for i in precip]
#     precip = precip[12:18]+precip[30:36]
    
#     swing = [float(temp_high[i])-float(temp_low[i]) for i in range(0,len(temp_high))]
#     avg_day = [(float(temp_high[i])+float(temp_low[i]))/2 for i in range(0,len(temp_high))]
#     ava = avg_day+swing+precip
#     df = pd.DataFrame(ava, index = ['Temp Jan','Temp Feb','Temp Mar','Temp Apr','Temp May','Temp Jun','Temp Jul','Temp Aug','Temp Sep','Temp Oct','Temp Nov','Temp Dec','Swing Jan','Swing Feb','Swing Mar','Swing Apr','Swing May','Swing Jun','Swing Jul','Swing Aug','Swing Sep','Swing Oct','Swing Nov','Swing Dec','Precip Jan','Precip Feb','Precip Mar','Precip Apr','Precip May','Precip Jun','Precip Jul','Precip Aug','Precip Sep','Precip Oct','Precip Nov','Precip Dec'])
#     df = df.transpose()
#     df['State'] = state
#     df['City'] = city
#     return precip
# get_weather('Florida','Apalachicola','https://www.usclimatedata.com/climate/atlanta/georgia/united-states/usga1761')

**Scrap weather parameters: NEW VERSION - pandas**

In [21]:
def get_weather(state, city, url):
    req = pd.read_html(url)
    janjun = req[0]
    juldec = req[1]
    janjun.rename(columns = {'Unnamed: 0':'Metric'}, inplace=True)
    juldec.rename(columns = {'Unnamed: 0':'Metric'}, inplace=True)
    
    snow, snow2, prec, prec2 = None, None, None, None
    
    for i, row in janjun.iterrows():
        if ("Average high" in row['Metric']):
            high = [i for i in row[1:]]
        if ("Average low" in row['Metric']):
            low = [i for i in row[1:]]
            
        if ("Av. precipitation" in row['Metric']):
            prec = [i for i in row[1:]]
            
        if ("snowfall in inch" in row['Metric']):
            snow = [i for i in row[1:]]
            
    for i, row in juldec.iterrows():
        if ("Average high" in row['Metric']):
            high2 = [i for i in row[1:]]
            
        if ("Average low" in row['Metric']):
            low2 = [i for i in row[1:]]
            
        if ("precipitation in inch" in row['Metric']):
            prec2 = [i for i in row[1:]]
            
        if ("snowfall in inch" in row['Metric']):
            snow2 = [i for i in row[1:]]

    
    if snow == None:
        snow=[0,0,0,0,0,0]
        snow2=[0,0,0,0,0,0]
    if prec == None:
        prec=[0,0,0,0,0,0]
        prec2=[0,0,0,0,0,0]
        
    
    highs = high+high2
    lows = low+low2
    precip = prec+prec2
    snows = snow+snow2
    
    highD = {"Jan High":highs[0],"Feb High":highs[1],"Mar High":highs[2],"Apr High":highs[3],"May High":highs[4],"Jun High":highs[5],"Jul High":highs[6],"Aug High":highs[7],"Sep High":highs[8],"Oct High":highs[9],"Nov High":highs[10],"Dec High":highs[11]}
    lowD = {"Jan Low":lows[0],"Feb Low":lows[1],"Mar Low":lows[2],"Apr Low":lows[3],"May Low":lows[4],"Jun Low":lows[5],"Jul Low":lows[6],"Aug Low":lows[7],"Sep Low":lows[8],"Oct Low":lows[9],"Nov Low":lows[10],"Dec Low":lows[11]}
    precipD = {"Jan Prec":precip[0],"Feb Prec":precip[1],"Mar Prec":precip[2],"Apr Prec":precip[3],"May Prec":precip[4],"Jun Prec":precip[5],"Jul Prec":precip[6],"Aug Prec":precip[7],"Sep Prec":precip[8],"Oct Prec":precip[9],"Nov Prec":precip[10],"Dec Prec":precip[11]}
    snowD = {"Jan Snow":snows[0],"Feb Snow":snows[1],"Mar Snow":snows[2],"Apr Snow":snows[3],"May Snow":snows[4],"Jun Snow":snows[5],"Jul Snow":snows[6],"Aug Snow":snows[7],"Sep Snow":snows[8],"Oct Snow":snows[9],"Nov Snow":snows[10],"Dec Snow":snows[11]}
    
    #
    combined=pd.DataFrame.from_dict({**highD,**lowD, **precipD, **snowD}, orient="index")
    combined = combined.transpose()
    combined['State'] = state
    combined['City'] = city
    return combined
get_weather('New York','Rochester','https://www.usclimatedata.com/climate/rochester/new-york/united-states/usny1232')

Jan High  Feb High  Mar High  Apr High  May High  Jun High  Jul High  \
0      32.0      34.0      43.0      56.0      68.0      77.0      81.0   

   Aug High  Sep High  Oct High  Nov High  Dec High  Jan Low  Feb Low  \
0      79.0      72.0      60.0      48.0      37.0     18.0     19.0   

   Mar Low  Apr Low  May Low  Jun Low  Jul Low  Aug Low  Sep Low  Oct Low  \
0     26.0     37.0     46.0     56.0     61.0     60.0     52.0     42.0   

   Nov Low  Dec Low  Jan Prec  Feb Prec  Mar Prec  Apr Prec  May Prec  \
0     33.0     24.0      2.41      1.95       2.5      2.73      2.87   

   Jun Prec  Jul Prec  Aug Prec  Sep Prec  Oct Prec  Nov Prec  Dec Prec  \
0      3.34      3.33      3.47      3.38      2.72      2.94      2.63   

   Jan Snow  Feb Snow  Mar Snow  Apr Snow  May Snow  Jun Snow  Jul Snow  \
0      28.0      22.0      16.0       4.0       0.0       0.0       0.0   

   Aug Snow  Sep Snow  Oct Snow  Nov Snow  Dec Snow     State       City  
0       0.0       0.0       0.0       7.0      22.0  New York  Rochester

**The next step takes a while, depending on your internet connection! Expect it to run somewhere around 30min!**

In [22]:
all_weather_df = pd.DataFrame()
for i, row in state_city_df.iterrows():
    try:
        newweather = get_weather(row.State, row.City, row.url_city)
        all_weather_df = all_weather_df.append(newweather, ignore_index=True)
    except:
        pass

In [23]:
print(all_weather_df.shape)
all_weather_df.head()

(0, 0)


Empty DataFrame
Columns: []
Index: []

**State abbreviations**

We are going to add the abbreviation to each state for future analytical purposes. These will become clear in the Data Collection - DATAUSA section.

---
To add the abbreviations, we must get a list of abbreviations and their respective states spelled out. Luckily, Wikipedia has a site listing all the US abbreviations! We will first import them, and then add the appropiate abbreviation to each City in our dataframe.

In [24]:
abb_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations', skiprows=11)[0]
abb_df = abb_df[['United States of America', 'Unnamed: 5']]
abb_df = abb_df.rename(columns={'United States of America':'State','Unnamed: 5':'State_Abb'})

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
#Now let's create a list of urls for further scrapping
weatherCity_df = all_weather_df
weatherCity_df = weatherCity_df.merge(abb_df, on='State')
weatherCity_df['CityState'] = [row.City+", "+row.State_Abb for i, row in weatherCity_df.iterrows()]

#Shift city and state column to firsts
cols = weatherCity_df.columns.tolist()
cols = cols[-4:]+cols[:-4]
weatherCity_df = weatherCity_df[cols]


In [ ]:
weatherCity_df.tail()

There it goes! We've scraped the weather information and structured it in our desired format. Now, lets go to the next section to scrap the general information for each city.

In [ ]:
weatherCity_df.shape

In [ ]:
weatherCity_df.to_csv("F:/0STUDY_2/Term6/Machine Learning/Proje/finding_schitts-master/Data/final_weather_data.csv")